In [1]:
import string
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_recommenders as tfrs
import json
from collections import Counter
from typing import Dict, Text
from ast import literal_eval
from datetime import datetime
from wordcloud import WordCloud
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

In [2]:
rating = pd.read_csv('./try/rating.csv')
fiction_metadata = pd.read_csv('./try/fiction_metadata.csv')

In [3]:
rating.head(10)

,user_id,fiction_id,click,like,rating
0,55,25,1,1,2
1,7,15,1,1,2
2,46,16,1,1,1
3,44,15,1,1,2
4,72,17,1,1,5
5,92,25,1,1,3
6,85,19,1,1,1
7,48,10,1,1,3
8,32,21,1,1,3
9,74,3,1,1,1


In [4]:
fiction_metadata.head(10)

,fiction_id,title,overview,language,release_date,latest_update,tags,genres,chapter
0,1,Kejahatan Cinta,"Seorang detektif swasta, Anya, sedang menyelid...",Indonesia,2023-10-01,2023-10-06,"detektif,pembunuhan,misteri,cinta,romansa","romansa,misteri",2
1,2,Raja Terakhir,"Di dunia yang dikuasai oleh iblis, seorang pem...",Indonesia,2023-10-02,2023-10-02,"fantasi,aksi,petualangan,pertarungan,kekuatan","fantasi,aksi",1
2,3,Perjalanan ke Masa Depan,Seorang ilmuwan muda menemukan cara untuk mela...,Indonesia,2023-10-03,2023-10-03,"fiksi ilmiah,petualangan,perjalanan waktu,mist...","sci-fi,petualangan",1
3,4,Cinta di Balik Tirai,Dua orang yang berasal dari latar belakang yan...,Indonesia,2023-10-04,2023-10-04,"romansa,drama,perbedaan latar belakang,rahasia...","romansa,drama",1
4,5,Pewaris Sang Dewa,Seorang pemuda yang terpilih sebagai pewaris s...,Indonesia,2023-10-05,2023-10-05,"fantasi,aksi,petualangan,dewa,takdir","fantasi,aksi",1
5,6,Legenda Tanah Jawa,Seorang pemuda yang hidup di masa kini menemuk...,Indonesia,2023-10-06,2023-10-06,"antasi,aksi,petualangan,sejarah,legenda","sci-fi,petualangan",1
6,7,Reinkarnasi,Seorang wanita yang meninggal dalam kecelakaan...,Indonesia,2023-10-07,2023-10-07,"reinkarnasi,keluarga,cinta,kehilangan","fantasi,drama",1
7,8,Penyelamat Dunia,Seorang remaja yang memiliki kekuatan super ha...,Indonesia,2023-10-08,2023-10-08,"sci-fi,aksi,petualangan,alien, penyelamatan dunia","sci-fi,petualangan",1
8,9,Cinta Tak Terduga,Dua orang yang awalnya tidak menyukai satu sam...,Indonesia,2023-10-09,2023-10-09,"romansa,komedi,persahabatan,sekolah","romansa,komedi",1
9,10,Kisah Cinta yang Tak Terlupakan,Sepasang kekasih yang harus terpisah karena su...,Indonesia,2023-10-10,2023-10-13,"romansa,drama,cinta pertama,reuni","romansa,drama",1


In [5]:
fiction = fiction_metadata[['fiction_id','title','overview','language','tags','genres','chapter']]

In [6]:
fiction.head(5)

,fiction_id,title,overview,language,tags,genres,chapter
0,1,Kejahatan Cinta,"Seorang detektif swasta, Anya, sedang menyelid...",Indonesia,"detektif,pembunuhan,misteri,cinta,romansa","romansa,misteri",2
1,2,Raja Terakhir,"Di dunia yang dikuasai oleh iblis, seorang pem...",Indonesia,"fantasi,aksi,petualangan,pertarungan,kekuatan","fantasi,aksi",1
2,3,Perjalanan ke Masa Depan,Seorang ilmuwan muda menemukan cara untuk mela...,Indonesia,"fiksi ilmiah,petualangan,perjalanan waktu,mist...","sci-fi,petualangan",1
3,4,Cinta di Balik Tirai,Dua orang yang berasal dari latar belakang yan...,Indonesia,"romansa,drama,perbedaan latar belakang,rahasia...","romansa,drama",1
4,5,Pewaris Sang Dewa,Seorang pemuda yang terpilih sebagai pewaris s...,Indonesia,"fantasi,aksi,petualangan,dewa,takdir","fantasi,aksi",1


In [7]:
rating['user_id'] = rating['user_id'].astype(int)
rating['fiction_id'] = rating['fiction_id'].astype(int)
rating['click'] = rating['click'].astype(int)
rating['like'] = rating['like'].astype(int)
rating['rating'] = rating['rating'].astype(int)

fiction['fiction_id'] = fiction['fiction_id'].astype(int)
fiction['overview'] = fiction['overview'].astype(str)
fiction['language'] = fiction['language'].astype(str)
fiction['tags'] = fiction['tags'].astype(str)
fiction['genres'] = fiction['genres'].astype(str)
fiction['chapter'] = fiction['chapter'].astype(str)

In [8]:
from scipy.sparse import coo_matrix

#menginisiasi ratings_mat_coo dengan sparse matrix dari kolom rating, user_index dan book_index
ratings_mat_coo = coo_matrix((rating["rating"], (rating["user_id"], rating["fiction_id"])))

In [9]:
#mengetahui ukuran dari matrix
ratings_mat_coo.shape

(101, 26)

In [10]:
#menginversikan coo matrix ke csr matrix
ratings_mat = ratings_mat_coo.tocsr()

In [11]:
rating[rating["user_id"] == 1]

,user_id,fiction_id,click,like,rating
15,1,20,1,1,1
31,1,16,1,1,2
134,1,23,1,1,5
318,1,1,1,1,3
367,1,16,1,1,3
729,1,6,1,1,3
881,1,17,1,1,2


In [12]:
my_index = 1

In [13]:
#menghitung similarity menggunakan cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(ratings_mat[my_index,:], ratings_mat).flatten()

In [14]:
similarity[2]

0.06919444144937047

In [15]:
similarity[3]

0.455179392247823

In [16]:
#menginisiasi indices sebagai 15 user yang paling mirip dengan kita
import numpy as np

indices = np.argpartition(similarity, -15)[-15:]

In [17]:
indices

array([63, 44, 60,  1, 87, 40, 37, 76, 14, 95, 10, 77, 79,  3, 50],
      dtype=int64)

In [18]:
#mengcopy user ke similar_user yang memiliki kemiripan
similar_users = rating[rating["user_id"].isin(indices)].copy()

In [19]:
#mengecek kemiripan dengan user id -1
similar_users = similar_users[similar_users["user_id"]!=1]

In [20]:
similar_users

,user_id,fiction_id,click,like,rating
3,44,15,1,1,2
14,87,2,1,1,5
16,95,11,1,1,4
17,79,22,1,1,4
18,37,17,1,1,1
...,...,...,...,...,...
974,3,6,1,1,5
976,44,16,1,1,2
979,95,21,1,1,5
981,10,2,1,1,4


In [26]:
#mencari berapa banyak buku tampil dalam rekomendasi dan mencari rata-rata ratings
fiction_recs = rating.groupby("fiction_id").rating.agg(['count', 'mean'])
fiction_recs['click'] = rating.groupby("fiction_id").click.agg(['count'])
fiction_recs['like'] = rating.groupby("fiction_id").like.agg(['count'])
fiction_recs['popularity'] = fiction_recs['click'] + fiction_recs['like']

In [27]:
fiction_recs

,count,mean,click,like,popularity
fiction_id,,,,,
1,41,2.853659,41,41,82
2,40,3.525000,40,40,80
3,32,2.625000,32,32,64
4,43,3.395349,43,43,86
5,44,3.295455,44,44,88
6,42,3.190476,42,42,84
7,39,2.615385,39,39,78
8,50,3.280000,50,50,100
9,45,3.244444,45,45,90


In [28]:
fiction = fiction.merge(fiction_recs, how="inner", on="fiction_id")

In [29]:
fiction.head()

,fiction_id,title,overview,language,tags,genres,chapter,count,mean,click,like,popularity
0,1,Kejahatan Cinta,"Seorang detektif swasta, Anya, sedang menyelid...",Indonesia,"detektif,pembunuhan,misteri,cinta,romansa","romansa,misteri",2,41,2.853659,41,41,82
1,2,Raja Terakhir,"Di dunia yang dikuasai oleh iblis, seorang pem...",Indonesia,"fantasi,aksi,petualangan,pertarungan,kekuatan","fantasi,aksi",1,40,3.525000,40,40,80
2,3,Perjalanan ke Masa Depan,Seorang ilmuwan muda menemukan cara untuk mela...,Indonesia,"fiksi ilmiah,petualangan,perjalanan waktu,mist...","sci-fi,petualangan",1,32,2.625000,32,32,64
3,4,Cinta di Balik Tirai,Dua orang yang berasal dari latar belakang yan...,Indonesia,"romansa,drama,perbedaan latar belakang,rahasia...","romansa,drama",1,43,3.395349,43,43,86
4,5,Pewaris Sang Dewa,Seorang pemuda yang terpilih sebagai pewaris s...,Indonesia,"fantasi,aksi,petualangan,dewa,takdir","fantasi,aksi",1,44,3.295455,44,44,88


In [30]:
R = fiction['mean']
v = fiction['count']
# We will only consider movies that have more votes than at least 80% of the movies in our dataset
m = fiction['count'].quantile(0.8)
C = fiction['mean'].mean()

fiction_recs['weighted_average'] = (R*v + C*m)/(v+m)

In [31]:
scaler = MinMaxScaler()
scaled = scaler.fit_transform(fiction_recs[['popularity', 'weighted_average']])
weighted_df = pd.DataFrame(scaled, columns=['popularity', 'weighted_average'])

weighted_df.index = fiction['fiction_id']

In [32]:
weighted_df.head(5)

,popularity,weighted_average
fiction_id,,
1,0.571429,1.000000
2,0.523810,0.345406
3,0.142857,0.912839
4,0.666667,0.836471
5,0.714286,0.748491


In [33]:
weighted_df['score'] = weighted_df['weighted_average']*0.4 + weighted_df['popularity'].astype('float64')*0.6

In [34]:
weighted_df_sorted = weighted_df.sort_values(by='score', ascending=False)
weighted_df_sorted.head(10)

,popularity,weighted_average,score
fiction_id,,,
8,1.000000,0.797860,0.919144
23,0.857143,0.598293,0.753603
9,0.761905,0.737338,0.752078
1,0.571429,1.000000,0.742857
4,0.666667,0.836471,0.734588
11,0.904762,0.476912,0.733622
5,0.714286,0.748491,0.727968
18,0.761905,0.638783,0.712656
21,0.761905,0.599449,0.696922


In [35]:
hybrid_df = fiction[['fiction_id','title','overview','language','tags','genres','chapter']]

In [36]:
hybrid_df.head(10)

,fiction_id,title,overview,language,tags,genres,chapter
0,1,Kejahatan Cinta,"Seorang detektif swasta, Anya, sedang menyelid...",Indonesia,"detektif,pembunuhan,misteri,cinta,romansa","romansa,misteri",2
1,2,Raja Terakhir,"Di dunia yang dikuasai oleh iblis, seorang pem...",Indonesia,"fantasi,aksi,petualangan,pertarungan,kekuatan","fantasi,aksi",1
2,3,Perjalanan ke Masa Depan,Seorang ilmuwan muda menemukan cara untuk mela...,Indonesia,"fiksi ilmiah,petualangan,perjalanan waktu,mist...","sci-fi,petualangan",1
3,4,Cinta di Balik Tirai,Dua orang yang berasal dari latar belakang yan...,Indonesia,"romansa,drama,perbedaan latar belakang,rahasia...","romansa,drama",1
4,5,Pewaris Sang Dewa,Seorang pemuda yang terpilih sebagai pewaris s...,Indonesia,"fantasi,aksi,petualangan,dewa,takdir","fantasi,aksi",1
5,6,Legenda Tanah Jawa,Seorang pemuda yang hidup di masa kini menemuk...,Indonesia,"antasi,aksi,petualangan,sejarah,legenda","sci-fi,petualangan",1
6,7,Reinkarnasi,Seorang wanita yang meninggal dalam kecelakaan...,Indonesia,"reinkarnasi,keluarga,cinta,kehilangan","fantasi,drama",1
7,8,Penyelamat Dunia,Seorang remaja yang memiliki kekuatan super ha...,Indonesia,"sci-fi,aksi,petualangan,alien, penyelamatan dunia","sci-fi,petualangan",1
8,9,Cinta Tak Terduga,Dua orang yang awalnya tidak menyukai satu sam...,Indonesia,"romansa,komedi,persahabatan,sekolah","romansa,komedi",1
9,10,Kisah Cinta yang Tak Terlupakan,Sepasang kekasih yang harus terpisah karena su...,Indonesia,"romansa,drama,cinta pertama,reuni","romansa,drama",1


In [37]:
def separate(text):
    clean_text = []
    for t in text.split(','):
        cleaned = re.sub('\(.*\)', '', t) # Remove text inside parentheses
        cleaned = cleaned.translate(str.maketrans('','', string.digits))
        cleaned = cleaned.replace(' ', '')
        cleaned = cleaned.translate(str.maketrans('','', string.punctuation)).lower()
        clean_text.append(cleaned)
    return ' '.join(clean_text)

def remove_punc(text):
    cleaned = text.translate(str.maketrans('','', string.punctuation)).lower()
    clean_text = cleaned.translate(str.maketrans('','', string.digits))
    return clean_text

In [38]:
hybrid_df['overview'] = hybrid_df['overview'].apply(remove_punc)
hybrid_df['language'] = hybrid_df['language'].apply(separate)
hybrid_df['tags'] = hybrid_df['tags'].apply(separate)
hybrid_df['genres'] = hybrid_df['genres'].apply(separate)

In [39]:
hybrid_df.head(10)

,fiction_id,title,overview,language,tags,genres,chapter
0,1,Kejahatan Cinta,seorang detektif swasta anya sedang menyelidik...,indonesia,detektif pembunuhan misteri cinta romansa,romansa misteri,2
1,2,Raja Terakhir,di dunia yang dikuasai oleh iblis seorang pemu...,indonesia,fantasi aksi petualangan pertarungan kekuatan,fantasi aksi,1
2,3,Perjalanan ke Masa Depan,seorang ilmuwan muda menemukan cara untuk mela...,indonesia,fiksiilmiah petualangan perjalananwaktu mister...,scifi petualangan,1
3,4,Cinta di Balik Tirai,dua orang yang berasal dari latar belakang yan...,indonesia,romansa drama perbedaanlatarbelakang rahasia c...,romansa drama,1
4,5,Pewaris Sang Dewa,seorang pemuda yang terpilih sebagai pewaris s...,indonesia,fantasi aksi petualangan dewa takdir,fantasi aksi,1
5,6,Legenda Tanah Jawa,seorang pemuda yang hidup di masa kini menemuk...,indonesia,antasi aksi petualangan sejarah legenda,scifi petualangan,1
6,7,Reinkarnasi,seorang wanita yang meninggal dalam kecelakaan...,indonesia,reinkarnasi keluarga cinta kehilangan,fantasi drama,1
7,8,Penyelamat Dunia,seorang remaja yang memiliki kekuatan super ha...,indonesia,scifi aksi petualangan alien penyelamatandunia,scifi petualangan,1
8,9,Cinta Tak Terduga,dua orang yang awalnya tidak menyukai satu sam...,indonesia,romansa komedi persahabatan sekolah,romansa komedi,1
9,10,Kisah Cinta yang Tak Terlupakan,sepasang kekasih yang harus terpisah karena su...,indonesia,romansa drama cintapertama reuni,romansa drama,1


In [40]:
hybrid_df['bag_of_words'] = ''
hybrid_df['bag_of_words'] = hybrid_df[hybrid_df.columns[1:]].apply(lambda x: ' '.join(x), axis=1)
hybrid_df.set_index(['fiction_id','title'], inplace=True)

In [41]:
hybrid_df = hybrid_df[['bag_of_words']]
hybrid_df.head(5)

,,bag_of_words
fiction_id,title,
1,Kejahatan Cinta,Kejahatan Cinta seorang detektif swasta anya s...
2,Raja Terakhir,Raja Terakhir di dunia yang dikuasai oleh ibli...
3,Perjalanan ke Masa Depan,Perjalanan ke Masa Depan seorang ilmuwan muda ...
4,Cinta di Balik Tirai,Cinta di Balik Tirai dua orang yang berasal da...
5,Pewaris Sang Dewa,Pewaris Sang Dewa seorang pemuda yang terpilih...


In [42]:
hybrid_df = weighted_df_sorted[:10000].merge(hybrid_df, left_index=True, right_index=True, how='left')

tfidf = TfidfVectorizer(stop_words='english', min_df=5)
tfidf_matrix = tfidf.fit_transform(hybrid_df['bag_of_words'])
tfidf_matrix.shape

(25, 24)

In [43]:
cos_sim = cosine_similarity(tfidf_matrix)
cos_sim.shape

(25, 25)

In [44]:
def predict(title, similarity_weight=0.9995, top_n=12_000):
    data = hybrid_df.reset_index()
    index_movie = data[data['title'] == title].index
    similarity = cos_sim[index_movie].T
    
    sim_df = pd.DataFrame(similarity, columns=['similarity'])
    final_df = pd.concat([data, sim_df], axis=1)
    # You can also play around with the number
    final_df['final_score'] = final_df['score']*(1-similarity_weight) + final_df['similarity']*similarity_weight
    
    final_df_sorted = final_df.sort_values(by='final_score', ascending=False).head(top_n)
    final_df_sorted.set_index('title', inplace=True)

    # Tambahkan kolom original_title
    final_df_sorted['title'] = final_df_sorted.index.values
    final_df_sorted['title'] = final_df_sorted['title'].astype(str)
    return final_df_sorted[['score', 'similarity', 'final_score', 'title']]

In [46]:
predict('Reinkarnasi').head(5)

,score,similarity,final_score,title
title,,,,
Reinkarnasi,0.621335,1.000000,0.999811,Reinkarnasi
Cahaya di Dalam Gelap,0.475319,0.673237,0.673138,Cahaya di Dalam Gelap
Keajaiban Cinta,0.269554,0.613143,0.612972,Keajaiban Cinta
Cinta di Balik Tirai,0.734588,0.574121,0.574201,Cinta di Balik Tirai
Perjalanan Menembus Dimensi,0.222963,0.566280,0.566108,Perjalanan Menembus Dimensi
